Neste trabalho iremos fazer a busca dos melhores hiperparametros de uma SVM para Regressão num banco de dados em particular. Para o desenvolvimento utilizaremos a biblioteca sklearn.svm que apresenta a função  SVR() que implementa o regressor SVM e tem vários hiperparametros.
Vamos usar o kernel “rbf”, havendo 3 hiperparametros que consideramos como os mais importantes: C, gamma, e epsilon.

Vamos fazer a busca no range:
    - C entre 2^{-5} e 2^{15}  (uniforme nos expoentes);
    - gamma entre 2^{15} e 2^{-3} (uniforme nos expoentes);
    - episolon entre 0.05 e 10  (uniforme neste intervalo);
    
Utilizamos como dados de treino e testes os arquivos Xtreino5.npy e Xteste5.npy. Para as sáidas dos dados correspondenetres usamos os arquivos ytreino5.npy e yteste5.npy. Todos os arquivos foram disponibilizados na página no trabalho https://www.ic.unicamp.br/~wainer/cursos/1s2020/431/ex4.html.

### Importação das Bibliotecas

Como primeiro passo, importamos as bibliotecas necessárias para a implementação desse trabalho: numpy e sklearn.svm.

In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import metrics
import pyspark
from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score

from simulated_annealing.optimize import SimulatedAnneal
from pyswarm import pso
from simanneal import Annealer
import cma


C:\Users\renat\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\renat\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


### Leitura e Exibição dos arquivos

Após importamos as bibliotecas necessárias, realizamos a leitura dos arquivos .npy disponibilizados para uso neste trabalho. Para isso fizemos uso da função load() da biblioteca numpy. Armazenamos cada um dos arquivos de treino e de teste em variáveis locais.

In [2]:
x_treino = np.load('data/Xtreino5.npy')
x_teste = np.load('data/Xteste5.npy')
y_treino = np.load('data/ytreino5.npy')
y_teste = np.load('data/yteste5.npy')

### Medida de erro

Como primeira tarefa, para cada conjunto de hiperparametros, treinamos o SVM no conjunto de treino (Xtreino e ytreino), e medimos o erro absoluto médio (MAE) no conjunto de teste (Xteste e yteste).

## Randomized Search

In [3]:
n_combinations = 125

# Hyperparameters Search Space
C_range = np.random.uniform(-5, 15, n_combinations).astype(float)
C_range = 2**C_range

gamma_range = np.random.uniform(-15, 3, n_combinations).astype(float)
gamma_range = 2**gamma_range

epsilon_range = np.random.uniform(0.05, 1.0, n_combinations).astype(float)

 
hyperparameters = {'gamma': list(gamma_range), 
                    'C': list(C_range),
                  'epsilon': list(epsilon_range)}
 
# print (hyperparameters)

In [ ]:
# Run randomized search
randomCV = RandomizedSearchCV(SVR(kernel='rbf'), param_distributions=hyperparameters, n_iter=20)
randomCV.fit(x_treino, y_treino)
 
# Identify optimal hyperparameter values
best_gamma  = randomCV.best_params_['gamma']
best_C      = randomCV.best_params_['C']
best_epsilon= randomCV.best_params_['epsilon']

print("The best performing C value is: {:5.2f}".format(best_C))
print("The best performing gamma value is: {:5.5f}".format(best_gamma))
print("The best performing epsilon value is: {:5.2f}".format(best_epsilon))


In [ ]:
# Validation

svr  = SVR(kernel='rbf', gamma=best_gamma, epsilon=best_epsilon, C=best_C)
svr.fit(x_treino, y_treino)

pred = svr.predict(x_teste)

# print(regression.score(x_teste, y_teste))
print("MAE: ", metrics.mean_absolute_error(y_true=y_teste, y_pred=pred))

### Grid seach

O próximo algoritimo de otimização proposto foi o Grid Search. 

In [ ]:
grid_c = C_range.reshape((5,5,5))
grid_gamma = gamma_range.reshape((5,5,5))
grid_episolon = epsilon_range.reshape((5,5,5))

hyperparameters_grid = [{'gamma': grid_gamma[0][0,:5], 'C': grid_c[0][0,:5], 'epsilon': grid_episolon[0][0,:5]},
                       {'gamma': grid_gamma[0][1,:5], 'C': grid_c[0][1,:5], 'epsilon': grid_episolon[0][1,:5]},
                       {'gamma': grid_gamma[0][2,:5], 'C': grid_c[0][2,:5], 'epsilon': grid_episolon[0][2,:5]},
                       {'gamma': grid_gamma[0][3,:5], 'C': grid_c[0][3,:5], 'epsilon': grid_episolon[0][3,:5]}]

In [ ]:
# Run randomized search
randomCV = GridSearchCV(SVR(kernel='rbf'), param_grid=hyperparameters_grid, cv = 5)
randomCV.fit(x_treino, y_treino)
 
# Identify optimal hyperparameter values
best_gamma  = randomCV.best_params_['gamma']
best_C      = randomCV.best_params_['C']
best_epsilon= randomCV.best_params_['epsilon']
 
print("The best performing gamma value is: {:5.5f}".format(best_gamma))
print("The best performing C value is: {:5.2f}".format(best_C))
print("The best performing epsilon value is: {:5.2f}".format(best_epsilon))

In [ ]:
# Validation

svr  = SVR(kernel='rbf', gamma=best_gamma, epsilon=best_epsilon, C=best_C)
svr.fit(x_treino, y_treino)

pred = svr.predict(x_teste)

# print(regression.score(x_teste, y_teste))
print("MAE: ", metrics.mean_absolute_error(y_true=y_teste, y_pred=pred))

### Otimização bayesiana

Outro algoritimo proposto foi a otimização bayesiana. Para a sua implementação utilziamos a biblioteca hyperopt, que dispnibiliza O regressor (TPE) para modelar a distribuição de probabilidades que é muito mais rápido que a implementação padrão utilizando “processos gaussianos”.

Esta implementação foi feita em passos:

### Definir a Função de mínimo

Como queremos pesquisar por Support Vector Machines (SVM), definimos um parâmetro params ['type'] como o nome do modelo, e uma função para executar o treinamento e retornar a precisão da validação cruzada. 
Como estamos tentando maximizar a precisão da validação cruzada, devemos negar esse valor para o hyperopt, pois o hyperopt sabe apenas como minimizar uma função.

In [ ]:
def get_acc_status(clf,X_,y):
    acc = cross_val_score(clf, X_, y, cv=5).mean()
    return {'loss': -acc, 'status': STATUS_OK}

In [ ]:
def objective(params):
    classifier_type = params['type']
    del params['type']
    if classifier_type == 'svm':
        clf = SVR(**params)
    else:
        return 0
    accuracy = cross_val_score(clf, x_treino, y_treino).mean()
    
    return {'loss': -accuracy, 'status': STATUS_OK}

### Definir espaço de pesquisa sobre os hiperparâmetros

In [ ]:
search_space = hp.choice('classifier_type', [
    {
        'type': 'svm',
        'C': hp.uniform('C', (2**-5), (2**15)),
        'gamma': hp.uniform('gamma', (2**-15), (2**3)),
        'epsilon': hp.uniform('epsilon', 0.05, 1.0),
        'kernel': hp.choice('kernel', ['rbf'])
    },
])

### Selecionar um algoritimo de busca

As duas opções principais de algoritmos de busca são:

    - hyperopt.tpe.suggest: Estimadores da Árvore de Parzen, uma abordagem bayesiana que seleciona iterativa e adaptativamente novas configurações de hiperparâmetro para explorar com base em resultados anteriores;
    - hyperopt.rand.suggest: Pesquisa aleatória, uma abordagem não adaptativa que mostra o espaço de pesquisa.
    
Conforme pedido, utilziamos o algoritimo TPE.

### Executar o algoritmo de ajuste com hyperopt fmin ()

Definimos max_evals como o número máximo de pontos no espaço do hiperparâmetro para testar, ou seja, o número máximo de modelos para ajustar e avaliar.

O SparkTrials usa 2 argumentos:
    - parallelism: Número de modelos para ajustar e avaliar simultaneamente.
    - timeout: tempo máximo (em segundos) que fmin pode demorar. Este argumento é opcional.

O rastreamento automatizado do MLflow está ativado por padrão. Ligue para mlflow.start_run () antes de chamar fmin (), como mostra o exemplo abaixo.

In [ ]:
hypopt_trials = Trials()
 
best_params = fmin(objective, search_space, algo=tpe.suggest, 
max_evals=125, trials= hypopt_trials)
 
best_gamma  = best_params['gamma']
best_C      = best_params['C']
best_epsilon= best_params['epsilon']
 
print("The best performing gamma value is: {:5.5f}".format(best_gamma))
print("The best performing C value is: {:5.2f}".format(best_C))
print("The best performing epsilon value is: {:5.2f}".format(best_epsilon))

In [ ]:
# Validation

svr  = SVR(kernel='rbf', gamma=best_gamma, epsilon=best_epsilon, C=best_C)
svr.fit(x_treino, y_treino)

pred = svr.predict(x_teste)

# print(regression.score(x_teste, y_teste))
print("MAE: ", metrics.mean_absolute_error(y_true=y_teste, y_pred=pred))

## PSO

In [ ]:
# PARAMETERS
C_MIN = 2**(-5)
C_MAX = 2**15

GAMMA_MIN = 2**(-15)
GAMMA_MAX = 2**3

EPSILON_MIN = 0.05
EPSILON_MAX = 1.0

lb = [C_MIN, GAMMA_MIN, EPSILON_MIN]
ub = [C_MAX, GAMMA_MAX, EPSILON_MAX]

# FUNCTION
def svr_fun(X):
    c = X[0]
    g = X[1]
    eps = X[2]
    
    svr  = SVR(kernel='rbf', C=c, gamma=g, epsilon=eps)
    svr.fit(x_treino, y_treino)
    
    pred = svr.predict(x_teste)
    mae = metrics.mean_absolute_error(y_true=y_teste, y_pred=pred)
    
    return mae

print("PSO...")
x_opt, y_opt = pso(svr_fun, lb, ub, swarmsize=11, maxiter=11)

print(" C optimal: "+ str(x_opt[0])+
     "\n Gamma Optimal: "+ str(x_opt[1])+
     "\n Epsilon Optimal: "+ str(x_opt[2]))
print("MAE: ", str(y_opt))

## Simulated Annealing

In [13]:
svr_params = {'C': C_range,
              'gamma': gamma_range,
              'epsilon': epsilon_range}

# Using a linear SVM classifier             
clf = SVR(kernel='rbf')
# Initialize Simulated Annealing and fit
sa = SimulatedAnneal(clf, svr_params, T=10.0, T_min=0.001, alpha=0.75,
                         verbose=True, max_iter=125, n_trans=5, max_runtime=300,
                         cv=3, scoring='f1_macro', refit=True)
sa.score(x_treino, y_treino)
# Print the best score and the best params
#print(sa.best_score_, sa.best_params_)


INFO: Number of possible iterations given cooling schedule: 160



AttributeError: 'SimulatedAnneal' object has no attribute 'score'

## CMA-ES